###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Almacena los datos en el formato requerido.

In [66]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Caso 1: Scraping de un catálogo: Labirratorium

In [76]:
URL = "https://www.labirratorium.com/es/67-cervezas-por-estilo?page="
response = requests.get(URL)
html = response.content

# Parsear el HTML con BeautifulSoup
contenido = BeautifulSoup(html, 'lxml')

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [174]:
# La web tiene XX páginas con XX cervezas listadas en cada página.
lista_paginas=[]
for i in range(1,50):
    lista_paginas.append(URL+str(i))
    
#Lista con todos los productos que existen la página
productos=[]

for pagina in lista_paginas:
    enlaces = contenido.find_all("a", class_="thumbnail product-thumbnail")
    for enlace in enlaces:
        primer_enlace = enlace['href']
        productos.append(primer_enlace)

In [163]:
for producto in productos:
    response = requests.get(producto)
    soup = BeautifulSoup(response.content, 'html.parser')
    nombre = soup.find_all('h1', {'class': 'product-detail-name'})
    descripcion = soup.find_all('div', {'class': 'product-description'})
    dic_productos['Nombre'].append(nombre[0].text.strip())
    dic_productos['Descripción'].append(descripcion[0].text.strip())


pd.DataFrame(dic_productos)



,Nombre,Descripción
0,SanFrutos Vikingathor Boris Brew,"Cerveza de 8.2% ABV intensa, oscura y muy lupu..."
1,Orval,Orval es una cerveza tipo ale y es la que prin...
2,SanFrutos Vikingathor Boris Brew,"Cerveza de 8.2% ABV intensa, oscura y muy lupu..."
3,"Boon Oude Kriek 37,5cl",Cerveza de fermentación espontánea (Lambic) de...
4,Störtebeker Schwarz-Bier,Cerveza negra de baja fermentación de estilo S...
...,...,...
585,Samuel Adams Boston Lager,Apariencia: color ambarino cobrizo y espuma pe...
586,Westmalle Dubbel,Cerveza Trapista belga con 7% ABV de color mar...
587,Duchesse De Bourgogne,Cerveza belga de estilo Ale Roja de 6.2% ABV d...
588,Tripel Karmeliet 33 cl.,Cerveza de estilo Belgian Tripel con 8.4% ABV....


### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

Construir un DataFrame con 

Nombre,	precio,	descripcion corta, descripción larga, ruta imagen, marca, barcode,	Estilo, Origen, % Alc., % Alc. Exacto, Volumen (Cl), Tipo Fermentación, IBU, IBU Exacto, Color, Envase

In [171]:
import requests
from bs4 import BeautifulSoup


data_beers = {"Nombre":[], "Descripcion":[], "Img":[], "Info":[]}

for producto in productos:
    response = requests.get(producto)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        nombre = soup.find_all('h1', {'class': 'product-detail-name'})
    except:
        nombre = ""
    try:
        descripcion = soup.find_all('div', {'class': 'product-description'})
    except:
        descripcion = ""
    try:
        img = soup.find_all('img', {'class': 'zoom_product'})
    except:
        img = ""
    try:
        info = soup.find_all('dl', {'class': 'datasheet'})
    except:
        info = ""

    data_beers['Nombre'].append(nombre[0].text.strip() if nombre else "")
    data_beers['Descripcion'].append(descripcion[0].text.strip() if descripcion else "")
    data_beers['Img'].append(img[0]['src'] if img else "")
    data_beers['Info'].append(info[0].text.strip() if info else "")

print(data_beers)



{'Nombre': ['SanFrutos Vikingathor Boris Brew', 'Boon Oude Kriek 37,5cl', 'Störtebeker Schwarz-Bier', 'Orval', 'Augustiner Lagerbier Hell', 'Schneider Aventinus Weizen-Eisbock', 'Aecht Schlenkerla Rauchbier Weizen', 'Samuel Adams Boston Lager', 'Westmalle Dubbel', 'Duchesse De Bourgogne', 'Tripel Karmeliet 33 cl.', 'Weihenstephaner Vitus', 'SanFrutos Vikingathor Boris Brew', 'Boon Oude Kriek 37,5cl', 'Störtebeker Schwarz-Bier', 'Orval', 'Augustiner Lagerbier Hell', 'Schneider Aventinus Weizen-Eisbock', 'Aecht Schlenkerla Rauchbier Weizen', 'Samuel Adams Boston Lager', 'Westmalle Dubbel', 'Duchesse De Bourgogne', 'Tripel Karmeliet 33 cl.', 'Weihenstephaner Vitus', 'SanFrutos Vikingathor Boris Brew', 'Boon Oude Kriek 37,5cl', 'Störtebeker Schwarz-Bier', 'Orval', 'Augustiner Lagerbier Hell', 'Schneider Aventinus Weizen-Eisbock', 'Aecht Schlenkerla Rauchbier Weizen', 'Samuel Adams Boston Lager', 'Westmalle Dubbel', 'Duchesse De Bourgogne', 'Tripel Karmeliet 33 cl.', 'Weihenstephaner Vitus'

In [173]:
a=pd.DataFrame(data_beers)
a.to_csv("prueba.csv")